In [1]:
# Custom imports
from Generate_Problem import generate_problem
from HHL_Circuit import hhl_circuit
from Iterative_Refinement import norm_estimation, sign_estimation, IR
#from Quantum_Linear_Solver import quantum_linear_solver

# Third-party libraries
import numpy as np
import pandas as pd

# Qiskit imports
import qiskit
import qiskit_aer

# PyTKet imports
import pytket
import qnexus as qnx
from pytket.extensions.nexus import NexusBackend
from pytket.extensions.nexus import QuantinuumConfig



# Filter warnings
# warnings.filterwarnings("ignore")

# Matplotlib inline to visualize plots
%matplotlib inline

# Use py310-qiskit1.0 environment

# Print library versions
print("Qiskit version (should be 1.3.0):", qiskit.__version__)
print("Qiskit Aer version (should be 0.15.1):", qiskit_aer.__version__)
print("Pytket version (should be 1.36.0):", pytket.__version__)

Qiskit version (should be 1.3.0): 1.4.2
Qiskit Aer version (should be 0.15.1): 0.16.1
Pytket version (should be 1.36.0): 1.40.0


In [2]:
problem = generate_problem(8, cond_number=5, sparsity=0.5, seed=1)
problem

{'A': array([[ 0.31, -0.  , -0.  ,  0.  ,  0.07, -0.02,  0.  , -0.  ],
        [-0.  ,  0.38, -0.  , -0.  , -0.  ,  0.03,  0.  , -0.  ],
        [-0.  , -0.  ,  0.21,  0.05, -0.  ,  0.  , -0.  , -0.  ],
        [ 0.  , -0.  ,  0.05,  0.42, -0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.07, -0.  , -0.  , -0.  ,  0.39,  0.  ,  0.  , -0.  ],
        [-0.02,  0.03,  0.  ,  0.  ,  0.  ,  0.42, -0.  ,  0.  ],
        [ 0.  ,  0.  , -0.  ,  0.  ,  0.  , -0.  ,  0.3 , -0.  ],
        [-0.  , -0.  , -0.  ,  0.  , -0.  ,  0.  , -0.  ,  0.32]]),
 'b': array([ 0.19, -0.07, -0.06, -0.12,  0.1 , -0.26,  0.2 , -0.09]),
 'csol': array([ 0.53921271, -0.13814447, -0.22403734, -0.25904317,  0.15962849,
        -0.58350336,  0.66666667, -0.28125   ]),
 'condition_number': 2.2325705187412592,
 'sparsity': 0.75,
 'eigs': array([0.19870297, 0.26730084, 0.3       , 0.32      , 0.36430171,
        0.42477906, 0.43129703, 0.44361839])}

In [3]:
# Create config object to specify details of desired backend
configuration = QuantinuumConfig(device_name='H2-2', attempt_batching=True)

# Set up a Nexus Project to compartmentalize running jobs and data
# Create a NexusBackend using our config and our retrieved project
backend = NexusBackend(configuration, project_name='HHL-IR')


Started using project with name: HHL-IR


In [6]:
from HHL_Circuit import hhl_circuit
import numpy as np
import math
from numpy import linalg as LA
from numpy.linalg import solve
from qiskit import transpile
from qiskit_aer import AerSimulator
from pytket.extensions.nexus import NexusBackend
from pytket.extensions.qiskit import qiskit_to_tk


def quantum_linear_solver_patched(A, b, backend, t0=2*np.pi, shots=1024):  # run hhl circuit on a quantum backend and post-process the result
    """
    Run the hhl circuit on a quantinuum backend and return the result and the compiled circuit.
    Returns:
    The post-processed result of the quantum linear solver (x), and a whole bunch of stats about the circuit.
    """
    csol = solve(A, b)
    solution = {}

    hhl_circ = hhl_circuit(A, b, t0)
    # transpile in qiskit then convert to a tket circuit if backend is from quantinuum
    qiskit_circuit = transpile(hhl_circ, AerSimulator())

    if isinstance(backend, NexusBackend):
        print(f"Running on {backend.backend_config.device_name}")
        qtuum_circuit = qiskit_to_tk(qiskit_circuit)
        # re-transpile to quantinuum backend
        new_qtuum_circuit = backend.get_compiled_circuit(qtuum_circuit, optimisation_level=2)

        try:
            syntax_checker = backend.backend_config.device_name[:4] + "SC"
            solution['cost'] = backend.cost(new_qtuum_circuit, shots, syntax_checker)
        except Exception:
            solution['cost'] = 0

        # Get circuit stats
        solution['number_of_qubits'] = new_qtuum_circuit.n_qubits
        solution['circuit_depth'] = new_qtuum_circuit.depth()
        solution['total_gates'] = new_qtuum_circuit.n_gates
        solution['two_qubit_gates'] = new_qtuum_circuit.n_2qb_gates()

        # Run circuit
        result_handle = ('2196ca2f-e935-4ade-b877-a028e6a6edd7', 3147224)
        solution['result_handle'] = result_handle

        result = backend.get_result(result_handle)  # only works if job completed
        status = backend.circuit_status(result_handle)

        try:
            solution['runtime'] = status.completed_time - status.running_time
        except Exception:
            # Skip the line if an error occurs
            solution['runtime'] = 'Not Found'

    else:
        print('backend should be a NexusBackend')

    def process_result(result):  # process the result of the quantum linear solver and return the solution vector
        """
        Process the result of the quantum linear solver and return the solution vector.
        """
        # Get the counts
        counts = result.get_counts()

        def solution_vector(counts, b):
            b_num = int(math.log2(len(b)))
            num = 0  # for normalization
            app_sol = np.zeros(2 ** b_num)

            if isinstance(backend, AerSimulator):
                for key, value in counts.items():
                    if key[-1] == '1':
                        num += value
                        cord = int(key[:b_num], base=2)  # position in b vector from binary string
                        app_sol[cord] = value
                if num == 0:
                    return app_sol
                app_sol = np.sqrt(app_sol/num)
                # app_sol = app_sol/num
                return app_sol

            else:
                for key, value in counts.items():
                    key_str = "".join(str(bit) for bit in key)
                    if key_str[-1] == '1':
                        num += value
                        cord = int(key_str[:b_num], base=2)
                        app_sol[cord] = value
                if num == 0:
                    return app_sol
                app_sol = np.sqrt(app_sol/num)
                # app_sol = app_sol/num
                return app_sol

        # Extract approximate solution
        qsol = solution_vector(counts, b)

        # Avoiding sign estimation for now
        for idx, (i, j) in enumerate(zip(csol, qsol)):
            if i < 0:
                qsol[idx] = -j
        return qsol

    x = process_result(result)
    solution['x'] = x

    two_norm_error = LA.norm(csol - x)
    solution['two_norm_error'] = two_norm_error

    residual_error = LA.norm(b - A @ x)
    solution['residual_error'] = residual_error

    return solution

In [7]:
A = problem['A']
b = problem['b']

# Single Run
solution = quantum_linear_solver_patched(A, b, backend=backend, t0=2*np.pi, shots=1024)
solution

Running on H2-2


ResourceFetchFailed: Failed to fetch resource with status code: 404, message: {"message":"Resource Not Found"}

In [8]:
result_handle = ('2196ca2f-e935-4ade-b877-a028e6a6edd7', 3147224)
status = backend.circuit_status(result_handle)
status 

CircuitStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='Circuit has completed.', error_detail=None, completed_time=datetime.datetime(2025, 3, 28, 9, 2, 37, 350370, tzinfo=datetime.timezone.utc), queued_time=datetime.datetime(2025, 3, 25, 23, 29, 7, 578087, tzinfo=datetime.timezone.utc), submitted_time=datetime.datetime(2025, 3, 25, 20, 28, 8, 950615, tzinfo=datetime.timezone.utc), running_time=datetime.datetime(2025, 3, 28, 9, 2, 5, 175728, tzinfo=datetime.timezone.utc), cancelled_time=None, error_time=None, queue_position=None)

In [9]:
result = backend.get_result(result_handle)

ResourceFetchFailed: Failed to fetch resource with status code: 404, message: {"message":"Resource Not Found"}

In [ ]:
# Iterative Refinement
refined_solution = IR(A, b, precision=1e-5, max_iter=3, backend=backend, plot=True)
refined_solution

In [ ]:
df = pd.DataFrame()
# Make a dataframe out of the results
datarow = {
    "Backend": backend.backend_config.device_name,
    "Problem Size": f"{len(b)} x {len(b)}",
    "A": problem["A"],
    "b": problem["b"],
    "Condition Number": problem["condition_number"],
    "Sparsity": problem["sparsity"],
    "Number of Qubits": solution["number_of_qubits"],
    "Circuit Depth": solution["circuit_depth"],
    "Total Gates": solution["total_gates"],
    "Two-Qubit Gates": solution["two_qubit_gates"],  # Uncomment if needed
    # "Runtime": solution["runtime"],  # Uncomment if needed
    "||x_c - x_q|| without IR": solution["two_norm_error"],
    "||x_c - x_q|| with IR": refined_solution["errors"][-1],
    "||Ax - b|| without IR": solution["residual_error"],
    "||Ax - b|| with IR": refined_solution["residuals"][-1],
    "Total Iterations of IR": refined_solution["total_iterations"],
    # "Total Cost": refined_solution["total_cost"],
    "Error list": refined_solution["errors"],
    "Residual list": refined_solution["residuals"],
}
df = pd.concat([df, pd.DataFrame([datarow])], ignore_index=True)

# Display the DataFrame without showing "Error list" and "Residual list"
df_display = df.drop(columns=["Error list", "Residual list", "A", "b"])

df_display.style.hide(axis="index").format(precision=6).set_caption(f"{backend.backend_config.device_name} Results")